# Business Cycle - Interest Rates

## Imports

In [67]:
import sys
import os
sys.path.append(os.path.abspath(".."))

import pmp_functions_v4 as pmp

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

path = "../../Data_Ryan"

## Data Setup & Cleaning

In [68]:
# --- Load Riskfree Rate ---
factors_data = pd.read_excel(
    f"{path}/Factors.xlsx",
    index_col = 0,
    parse_dates = True
)

factors_data.index = pd.to_datetime(factors_data.index, format='%Y%m')
factors_data.index = factors_data.index + pd.offsets.MonthEnd(0)
factors_data /= 100

riskfree = factors_data["RF"].resample('ME').last()
display(riskfree)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_54746/711168680.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  factors_data = pd.read_excel(


1926-07-31    0.0022
1926-08-31    0.0025
1926-09-30    0.0023
1926-10-31    0.0032
1926-11-30    0.0031
               ...  
2025-06-30    0.0034
2025-07-31    0.0034
2025-08-31    0.0038
2025-09-30    0.0033
2025-10-31    0.0037
Freq: ME, Name: RF, Length: 1192, dtype: float64

In [69]:
# --- Load Factors Data ---
famafrench_data = pd.read_csv(
    f"{path}/famafrench_factors.csv",
    index_col = 0,
    parse_dates = True
)

famafrench_data.index = pd.to_datetime(famafrench_data.index, format='%Y%m')
famafrench_data.index = famafrench_data.index + pd.offsets.MonthEnd(0)
famafrench_data /= 100
display(famafrench_data)

/var/folders/bl/ywj9s7j57_qf7pwls_sv_w9r0000gn/T/ipykernel_54746/2041547973.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  famafrench_data = pd.read_csv(


,MKT-RF,SMB,HML,RMW,CMA,UMD,BAB
DATE,,,,,,,
1980-01-31,0.000550,0.000188,0.000185,-0.000184,0.000189,0.000745,0.000695
1980-02-29,-0.000123,-0.000162,0.000059,-0.000095,0.000292,0.000789,-0.000132
1980-03-31,-0.001289,-0.000697,-0.000096,0.000182,-0.000105,-0.000958,-0.001181
1980-04-30,0.000396,0.000105,0.000103,-0.000218,0.000034,-0.000048,0.000574
1980-05-31,0.000526,0.000200,0.000038,0.000043,-0.000063,-0.000118,0.000618
...,...,...,...,...,...,...,...
2025-06-30,0.000486,-0.000002,-0.000160,-0.000320,0.000145,-0.000264,0.000527
2025-07-31,0.000198,-0.000015,-0.000127,-0.000029,-0.000208,-0.000096,0.000184
2025-08-31,0.000185,0.000488,0.000442,-0.000068,0.000207,-0.000354,0.000646


In [70]:
# --- Benchmark Data ---
benchmark_data = pd.read_excel(
    f"{path}/Benchmarks.xlsx",
    index_col = 0,
    parse_dates = True
)
display(benchmark_data)
benchmark_data.index = pd.to_datetime(benchmark_data.index)
benchmark_data = benchmark_data.resample('ME').last()

benchmark_TR = benchmark_data[['MSCI World']].pct_change()
benchmark_XR = benchmark_TR.sub(riskfree, axis = 0)

benchmark_TR.columns = ['Benchmark Total Return']
benchmark_XR.columns = ['Benchmark Excess Return']
benchmark_returns = pd.concat([benchmark_TR, benchmark_XR], axis = 1).dropna()

display(benchmark_returns)

,S&P 500,MSCI World
Date,,
1986-12-31,242.1700,NaN
1987-01-30,274.7800,NaN
1987-02-27,285.6377,NaN
1987-03-31,293.8792,NaN
1987-04-30,291.2698,NaN
...,...,...
2025-07-31,14412.5527,8214.1572
2025-08-29,14704.7217,8431.0801
2025-09-30,15240.0381,8705.7139


,Benchmark Total Return,Benchmark Excess Return
1991-01-31,0.034364,0.029164
1991-02-28,0.090450,0.085650
1991-03-31,-0.031331,-0.035731
1991-04-30,0.005910,0.000610
1991-05-31,0.020740,0.016040
...,...,...
2025-06-30,0.043488,0.040088
2025-07-31,0.013121,0.009721
2025-08-31,0.026408,0.022608
2025-09-30,0.032574,0.029274


In [71]:
# --- Load Macro Data ---
CPI_forecasts = pd.read_excel(
    f"{path}/Inflation_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
CPI_forecasts.index = pd.to_datetime(CPI_forecasts.index)
CPI_forecasts.index = CPI_forecasts.index + pd.offsets.MonthEnd(0)
CPI_forecasts *= 100

display(CPI_forecasts)

RGDP_forecasts = pd.read_excel(
    f"{path}/RGDP_forecasts.xlsx",
    index_col = 0,
    parse_dates = True
)
RGDP_forecasts.index = pd.to_datetime(RGDP_forecasts.index)
RGDP_forecasts.index = RGDP_forecasts.index + pd.offsets.MonthEnd(0)
RGDP_forecasts *= 100

display(RGDP_forecasts)

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,4.95739,2.31338,7.75127,NaN,NaN,NaN
1970-02-28,4.93065,2.12360,7.75127,NaN,NaN,NaN
1970-03-31,5.14198,2.49918,7.71710,NaN,NaN,NaN
1970-04-30,5.61884,2.59281,7.68362,NaN,NaN,NaN
1970-05-31,6.08365,3.13683,7.24558,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,4.10000,0.10000,3.30000,2.1,1.96760,3.17
2025-07-31,4.20000,0.20000,3.10000,3.2,2.00791,2.79
2025-08-31,4.10000,0.20000,2.70000,3.2,2.02889,2.79


,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-08-31,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-09-30,3.15789,NaN,NaN,NaN,NaN,2.936175
1970-10-31,5.26317,NaN,NaN,NaN,NaN,2.994250
1970-11-30,5.26317,NaN,NaN,NaN,NaN,2.994250
...,...,...,...,...,...,...
2025-06-30,2.10000,2.5,2.439516,1.976375,1.522007,1.345350
2025-07-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175
2025-08-31,3.20000,2.5,1.284580,1.074700,1.350512,1.450175


In [72]:
# --- Load Interest Rate Futures ---
interest_futures = pd.read_excel(
    f"{path}/Interest Rates Data.xlsx",
    index_col = 0,
    parse_dates = True
)
interest_futures.index = pd.to_datetime(interest_futures.index)
interest_futures.index = interest_futures.index + pd.offsets.MonthEnd(0)

display("Interest Rate Futures:")
display(interest_futures)

'Interest Rate Futures:'

,CH,EU,AU,US,EM,UK,JP
Date,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,94.564,NaN,97.169,95.44,77.41,NaN,98.3323
1998-03-31,94.334,NaN,97.251,95.49,78.42,NaN,98.4613
1998-04-30,94.286,NaN,97.180,95.47,79.49,NaN,98.4910
1998-05-31,94.286,NaN,97.221,95.40,78.30,NaN,98.5456
...,...,...,...,...,...,...,...
2025-07-31,100.732,98.015,96.680,99.27,97.94,97.65,99.4600
2025-08-31,100.732,97.970,96.660,99.27,96.99,97.38,99.4600
2025-09-30,100.732,97.975,96.490,99.27,96.60,97.26,99.4600


## Global Variables

In [73]:
frequency = 1
t_cost = 0
# window = 12*20
short = True
beta_neutral = False
target_vol = 0.10
rf = riskfree
benchmark = benchmark_data

## Signal Generation

In [74]:
# --- Compute Business Cycle Signal ---
CPI_component = CPI_forecasts.diff(12)
RGDP_component = RGDP_forecasts.diff(12)

display(CPI_component)
display(RGDP_component)

# # Reindex RGDP trend to match CPI trend index exactly
# RGDP_component = RGDP_trend.reindex(CPI_trend.index).ffill()
# CPI_component = CPI_trend

,UK,CH,JP,AU,EU,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.31793,-1.22807,0.5,-1.71445,-0.54354,0.39
2025-07-31,1.17674,-1.09059,0.3,0.39146,-0.57075,0.39
2025-08-31,1.00881,-0.86037,-0.3,0.39146,-0.12434,0.39


,AU,UK,CH,JP,EU,US
Date,,,,,,
1970-07-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-08-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-09-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-10-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-11-30,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.71445,0.7,2.356965,2.957250,0.993241,-0.529850
2025-07-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325
2025-08-31,0.39146,1.7,-0.644766,0.676316,0.386473,-0.337325


In [75]:
# --- Composite Signal Construction ---
# Logic: For Interest Rates, we want Low Growth (-) and Low Inflation (-)
# We combine them: 50% Growth, 50% Inverse Inflation
business_cyle_signal = - (0.5 * RGDP_component) - (0.5 * CPI_component)
business_cyle_signal = business_cyle_signal.resample('ME').last()

display("Business Cycle Signal:")
display(business_cyle_signal)

'Business Cycle Signal:'

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.71445,-0.564448,-0.224851,-1.728625,-1.008965,0.069925
2025-07-31,-0.39146,0.867678,0.092139,-0.488158,-1.438370,-0.026338
2025-08-31,-0.39146,0.752568,-0.131066,-0.188158,-1.354405,-0.026338


## Asset Returns

In [76]:
# --- Interest Rate Returns ---
interest_XR = interest_futures.pct_change()
interest_XR = interest_XR.resample('ME').last()

display(interest_XR)

,CH,EU,AU,US,EM,UK,JP
Date,,,,,,,
1998-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1998-03-31,-0.002432,NaN,0.000844,0.000524,0.013047,NaN,0.001312
1998-04-30,-0.000509,NaN,-0.000730,-0.000209,0.013644,NaN,0.000302
1998-05-31,0.000000,NaN,0.000422,-0.000733,-0.014970,NaN,0.000554
...,...,...,...,...,...,...,...
2025-07-31,0.000000,-0.001019,-0.002281,0.000000,0.000000,-0.001840,0.000000
2025-08-31,0.000000,-0.000459,-0.000207,0.000000,-0.009700,-0.002765,0.000000
2025-09-30,0.000000,0.000051,-0.001759,0.000000,-0.004021,-0.001232,0.000000


## Portfolio Construction

In [77]:
# --- Ranking & Weighting ---
# Rank countries 1 to N for each month based on the raw signal.
# axis = 1 means we rank across columns (countries).

rank = business_cyle_signal.rank(axis = 1, method = 'average', ascending = False)
display(rank)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,1.0,4.0,3.0,6.0,5.0,2.0
2025-07-31,4.0,1.0,2.0,5.0,6.0,3.0
2025-08-31,5.0,1.0,3.0,4.0,6.0,2.0


In [78]:
# --- Standardize Ranks ---
# Convert ranks into Z-scores (Weights) that sum to zero.
# Weight = (Rank - Mean_Rank) / Std_Dev_Rank

rank_mean = rank.mean(axis = 1)
rank_stds = rank.std(axis = 1)
standardized_weights = rank.sub(rank_mean, axis = 0).div(rank_stds, axis = 0)

display(standardized_weights)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1970-01-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-02-28,NaN,NaN,NaN,NaN,NaN,NaN
1970-03-31,NaN,NaN,NaN,NaN,NaN,NaN
1970-04-30,NaN,NaN,NaN,NaN,NaN,NaN
1970-05-31,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...
2025-06-30,-1.336306,0.267261,-0.267261,1.336306,0.801784,-0.801784
2025-07-31,0.267261,-1.336306,-0.801784,0.801784,1.336306,-0.267261
2025-08-31,0.801784,-1.336306,-0.267261,0.267261,1.336306,-0.801784


In [79]:
# --- Volatility Scaling (Risk Management) ---
# Step A: Calculate 'Raw' Strategy Returns (Before Vol Scaling)
# IMPORTANT: Shift weights by 1 to trade next month's return.
strategy_raw_ret = (standardized_weights.shift(1) * interest_XR).sum(axis=1)

# Step B: Forecast Volatility
# Calculate realized volatility over a 36-month rolling window (annualized)
# We use the raw strategy's realized vol to estimate future volatility.
expected_vol = strategy_raw_ret.rolling(window = 36).std() * np.sqrt(12)

# We use previous rolling volatility (shift 1) to size today's position
lev_factor = target_vol / expected_vol.shift(1)

## **⭐ CRITICAL CHANGE: Scaling the Weights**

# Step C: Estimate Portfolio Weights
# Apply the leverage factor to the standardized weights
# We use .mul(axis=0) to multiply the 2D DataFrame (weights) 
# by the 1D Series (lev_factor_series) along the rows (axis=0).
final_strategy_weights = standardized_weights.mul(lev_factor, axis=0).fillna(0).loc['1998-04-30':'2025-10-31']

display(final_strategy_weights)

,AU,CH,EU,JP,UK,US
Date,,,,,,
1998-04-30,20.787743,41.575485,0.000000,-41.575485,0.000000,-20.787743
1998-05-31,20.774746,41.549492,0.000000,-41.549492,0.000000,-20.774746
1998-06-30,20.771181,41.542362,0.000000,-41.542362,0.000000,-20.771181
1998-07-31,35.041820,17.520910,0.000000,-35.041820,0.000000,-17.520910
1998-08-31,33.739717,16.869859,-16.869859,-33.739717,0.000000,0.000000
...,...,...,...,...,...,...
2025-06-30,-3.584601,0.716920,-0.716920,3.584601,2.150761,-2.150761
2025-07-31,0.718404,-3.592019,-2.155211,2.155211,3.592019,-0.718404
2025-08-31,2.180912,-3.634854,-0.726971,0.726971,3.634854,-2.180912


## Backtest

In [80]:
benchmark_TR = benchmark_TR.squeeze()
benchmark_XR = benchmark_XR.squeeze()

In [81]:
results = pmp.run_cc_strategy(
    weights = final_strategy_weights,
    returns = interest_XR,
    rf = riskfree,
    frequency = frequency,
    t_cost = t_cost, 
    benchmark = benchmark_XR,
    long_short = short,
    beta_neutral = beta_neutral
)

display(results)

,ret_net,ret_gross,ret_bm,turnover,tcost,ret_rf,w_AU,w_CH,w_EU,w_JP,w_UK,w_US
Date,,,,,,,,,,,,
1998-05-31,0.000964,0.000964,-0.015149,0.000000,0.0,0.0040,0.333426,0.666574,0.000000,-0.666951,0.000000,-0.333049
1998-06-30,-0.115245,-0.115245,0.020679,61.324238,0.0,0.0041,0.333748,0.666252,0.000000,-0.666761,0.000000,-0.333239
1998-07-31,0.050039,0.050039,-0.005774,61.313544,0.0,0.0040,0.333483,0.666517,0.000000,-0.666574,0.000000,-0.333426
1998-08-31,-0.109367,-0.109367,-0.137443,51.562731,0.0,0.0043,0.664718,0.335282,0.000000,-0.666490,0.000000,-0.333510
1998-09-30,0.292618,0.292618,0.013398,49.943086,0.0,0.0046,0.668433,0.331567,-0.333635,-0.666365,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
2025-06-30,0.010932,0.010932,0.040088,5.470249,0.0,0.0034,-0.555527,0.110699,-0.111250,0.554621,0.334680,-0.333223
2025-07-31,0.004949,0.004949,0.009721,5.452282,0.0,0.0034,-0.554970,0.111094,-0.111237,0.555982,0.332924,-0.333794
2025-08-31,-0.009091,-0.009091,0.022608,6.131698,0.0,0.0038,0.111418,-0.555797,-0.333231,0.333853,0.554729,-0.110972


## Performance Statistics

In [82]:
pmp.run_perf_summary_benchmark_vs_strategy(results, alreadyXs = True)

,Benchmark,Strategy
Arithm Avg Total Return,6.6556,1.6900
Arithm Avg Xs Return,4.6422,-0.3235
Std Xs Returns,15.5692,15.2113
Sharpe Arithmetic,0.2982,-0.0213
Geom Avg Total Return,5.5714,0.5179
Geom Avg Xs Return,3.5410,-1.5125
Sharpe Geometric,0.2274,-0.0994
Min Xs Return,-19.0940,-30.4919
Max Xs Return,12.8084,29.2618
Skewness,-0.5976,0.0772
